In [7]:
'''Linear programming solver using SciPy linprog function'''
# Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog

import csv
import time
import numpy as np
from scipy.optimize import linprog

# scale the cutoff sand amount given the size of Nbox (base of 15)
def setCutoffValue(Nbox):
    cutoff = 1e-4 * ((Nbox/15)**(-2))
    print("The scaled cutoff value is: ", cutoff)
    return cutoff

# convert the left side of constraint euqations into matrix form
def consMatrix(cArray, vList):
    consM = [];
    for i in range(len(cArray)):
        if cArray[i][-1] != 0:
            tempList = []
            for j in range(len(vList)):
                n = cArray[i].count(vList[j])
                tempList.append(n)
            consM.append(tempList)
    return consM

# convert the right side of constraint equations (the results) into matrix form
def resMatrix(cArray):
    resM = [];
    for i in range(len(cArray)):
        if cArray[i][-1] != 0:
            resM.append(cArray[i][-1])
    return resM

def EMD(position, w_diff, cutoff):

    index_of_inboxes = [] # store the index for positions that take in sand
    index_of_outboxes = [] # store the index for positions that give out sand

    for i in range(len(w_diff)):
        if (w_diff[i] > cutoff):
            index_of_outboxes.append(i)
        elif (w_diff[i] <= -cutoff):
            index_of_inboxes.append(i)
        
    I = len(index_of_inboxes); # number of positions that take in sand
    O = len(index_of_outboxes); # number of positions that give out sand
    
    # store the weight difference fulfill the cutoff requirement
    w = []
    for i in index_of_inboxes: # add in the sand amount differences for inbox positions
        w.append(w_diff[i]) 
    for j in index_of_outboxes: # add in the sand amount differences for outbox positions  
        w.append(w_diff[j])

    # define the euclidean distances between two distributions
    distances = np.zeros((I, O))
    for i in range(I):
        pi = index_of_inboxes[i]
        for j in range(O):
            pf = index_of_outboxes[j]
            distances[i][j] = np.linalg.norm(np.array(position[pi]) - np.array(position[pf]))
    
    # Set objective function: Minimize the summation of Xi_j * Di_j
    disArray = []
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            disArray.append(distances[i][j])
            
    c = np.array(disArray)
    
    #print("Objective funciton: ", c)
    
    # Set variables
    variablesList = []
    vList = []
    
    for i in range(I):
        tempList = []
        for j in range(O):
            vList.append("x"+str(index_of_outboxes[j])+"_"+str(index_of_inboxes[i]))
            tempList.append("x"+str(index_of_outboxes[j])+"_"+str(index_of_inboxes[i]))
        variablesList.append(tempList)
        
    for i in range(O):
        tempList = []
        for j in range(I):
            tempList.append("x"+str(index_of_outboxes[i])+"_"+str(index_of_inboxes[j]))
        variablesList.append(tempList)
        
    #print("List of variables: ", vList)
    
    # Set constraints
    consArray = [];
    for i in range(I+O-1): 
        cons = variablesList[i];
        cons.append(abs(w[i]))
        consArray.append(cons)
    
    #print("Arrays that contain all set constraints: ", consArray)
    
    ans = consMatrix(consArray,vList)
    
    #print("Left side of constraint equations: ", ans)
    
    res = resMatrix(consArray)
    
    #print("Right side of constraint equations", res)
    
    A = np.array(ans)
    b = np.array(res)
    
    # Set bounds for all variables to be greater or equal to zero
    options = {"disp": False, "maxiter": 500000}
    EMDresult = linprog(c, A_eq=A, b_eq=b, bounds=(0,None), method='revised simplex', options=options )
    
    return EMDresult

if __name__ == '__main__':
    
    t1 = time.process_time();
    
    #creat empty list to take in weights from before and after condition
    x_val = []
    y_val = []
    w_val = []

    #import data from corresponding csv document
    with open('Gaussians.csv') as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            x_val.append(row[0])
            y_val.append(row[1])
            w_val.append(row[2])
    
    #convert the list of string into float
    position_x = [float(i) for i in x_val]
    position_y = [float(i) for i in y_val]
    position = np.column_stack((position_x,position_y))
        
    w_diff = [float(i) for i in w_val]
    
    Nbox = np.sqrt(len(w_diff))
    cutoff = setCutoffValue(Nbox)
    
    result = EMD(position, w_diff, cutoff)
    
    t2 = time.process_time();

    print("\n", result)
    
    print("Process time for EMD calculation: ", (t2-t1), " seconds")
    

The scaled cutoff value is:  1.4062500000000003e-05

      con: array([ 0.00000000e+00,  3.38813179e-21,  0.00000000e+00, -1.35525272e-20,
        4.06575815e-20,  6.77626358e-21,  0.00000000e+00,  0.00000000e+00,
        5.42101086e-20,  4.06575815e-20,  1.49077799e-19,  2.71050543e-20,
        0.00000000e+00, -6.77626358e-21,  1.08420217e-19, -2.16840434e-19,
       -6.50521303e-19,  2.71050543e-20,  0.00000000e+00,  6.77626358e-20,
        1.62630326e-19, -8.67361738e-19,  3.25260652e-19,  1.35525272e-19,
       -6.77626358e-20,  0.00000000e+00,  6.77626358e-20, -1.62630326e-19,
        1.73472348e-18,  1.62630326e-18, -6.50521303e-19, -9.48676901e-20,
        0.00000000e+00,  0.00000000e+00, -1.35525272e-19, -2.16840434e-19,
       -8.67361738e-19, -8.67361738e-19, -4.33680869e-19,  2.71050543e-20,
       -6.77626358e-21,  0.00000000e+00, -1.62630326e-19, -1.19262239e-18,
       -2.16840434e-19, -1.95156391e-18,  2.16840434e-18,  7.58941521e-19,
       -1.35525272e-20, -2.03287907e